In [1]:
# ## install finrl library
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.20
    Uninstalling SQLAlchemy-2.0.20:
      Successfully uninstalled SQLAlchemy-2.0.20
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.0 MB/s eta 0:00:00
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...
Selecting previously unselected package libgl1-mesa-glx:amd64.
(Reading database ... 1

In [8]:
import pandas as pd
import numpy as np
import gym
from gym.spaces import Box, Discrete
from stable_baselines3 import DQN
from stable_baselines3.dqn import MlpPolicy

# Step 1: Load and Prepare Data
data = pd.read_csv('/content/Finsearch_dataset.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Step 2: Define Action Space
num_actions = 3
action_space = Discrete(num_actions)

# Step 3: Create Reward Function
def reward_function(portfolio_value, new_portfolio_value):
    return new_portfolio_value - portfolio_value

# Step 4: Define Custom Trading Environment
class CustomTradingEnv(gym.Env):
    def __init__(self, data, action_space, reward_function):
        self.data = data
        self.action_space = action_space
        self.reward_function = reward_function
        self.current_step = 0
        self.portfolio_value = 100000
        self.done = False
        self.info = {}

        # Define observation space based on closing prices
        self.observation_space = Box(low=0, high=np.inf, shape=(6,))  # Adjust shape based on your observation

    def reset(self):
        self.current_step = 0
        self.portfolio_value = 100000
        self.done = False
        self.info = {}
        return self.data.iloc[self.current_step]

    def step(self, action):
        if self.done:
            raise ValueError("Episode has ended. Call reset() to start a new episode.")

        current_price = self.data.iloc[self.current_step]['Close']
        new_portfolio_value = np.random.uniform(90000, 110000)
        reward = self.reward_function(self.portfolio_value, new_portfolio_value)

        self.portfolio_value = new_portfolio_value
        self.current_step += 1
        next_observation = self.data.iloc[self.current_step]
        self.done = self.current_step == len(self.data) - 1

        return next_observation, reward, self.done, self.info

# Step 5: Create the custom trading environment
env = CustomTradingEnv(data, action_space, reward_function)

# Step 6: Choose DQN algorithm and train the agent
model = DQN(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=int(1e4))

# Step 7: Evaluate the Agent
def evaluate_agent(model, env, num_episodes=10):
    episode_rewards = []

    for episode in range(num_episodes):
        obs = env.reset()
        done = False
        total_reward = 0

        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            total_reward += reward

        episode_rewards.append(total_reward)
        print(f"Episode {episode + 1}: Total Reward = {total_reward}")

    average_reward = sum(episode_rewards) / num_episodes
    print(f"Average Reward over {num_episodes} episodes: {average_reward}")

# Usage
testing_data = pd.read_csv('/content/Finsearch_dataset.csv')
testing_data['Date'] = pd.to_datetime(testing_data['Date'])
testing_data.set_index('Date', inplace=True)
testing_env = CustomTradingEnv(testing_data, action_space, reward_function)
evaluate_agent(model, testing_env, num_episodes=10)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 2.34e+03  |
|    ep_rew_mean      | -2.36e+03 |
|    exploration_rate | 0.05      |
| time/               |           |
|    episodes         | 4         |
|    fps              | 2004      |
|    time_elapsed     | 4         |
|    total_timesteps  | 9380      |
-----------------------------------
Episode 1: Total Reward = -892.7855738754879
Episode 2: Total Reward = 1805.5655665049417
Episode 3: Total Reward = -8817.68002411208
Episode 4: Total Reward = 9946.99103901643
Episode 5: Total Reward = 7170.14128628849
Episode 6: Total Reward = 5277.647664647884
Episode 7: Total Reward = 105.02562596437929
Episode 8: Total Reward = 6252.22703116326
Episode 9: Total Reward = -5455.634899955374
Episode 10: Total Reward = 9451.66302380785
Average Reward over 10 episodes: 2484.3160739450295
